In [1]:
import os

import cv2
from IPython.display import display,clear_output
import ultralytics
from ultralytics import YOLO
import torch
import shapely
import chess
import time
from shapely.geometry import Polygon, Point
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import subprocess
import chess.svg
from IPython.display import SVG
import chess
import cv2
import numpy as np
from PIL import ImageFont, ImageDraw, Image
import platform

from stockfish_downloader import download_stockfish
download_stockfish()

Detected OS: Darwin, Architecture: arm64
Stockfish is already downloaded. path: /Users/akhil838/Documents/PyCharm_Projects/MiniProject/stockfish


In [2]:
# model = YOLO('./runs/detect/train22/weights/best.pt')
model = YOLO('model/best.pt')

In [3]:
# cap = cv2.VideoCapture(1)
# while cap.isOpened():
#     ret, frame = cap.read()  # Read frame from webcam
#     if not ret:
#         break
#
#     # Run YOLO inference
#     results = model.predict(frame ,conf = 0.3, device="mps",verbose = False) # Use GPU if available
#     # Visualize results on frame
#     for result in results:
#         if result.obb is None or len(result.obb) == 0:
#             display("No oriented bounding boxes detected")
#             continue  # Skip frame if no OBBs
#
#         for obb in result.obb:
#             if obb is None or obb.xyxyxyxy is None:
#                 continue  # Skip invalid OBBs
#
#             # Extract 4 corner points from OBB
#             points = obb.xyxyxyxy.cpu().numpy().astype(int)  # Convert tensor to numpy array
#
#             # Draw OBB polygon
#             points = points.reshape((-1, 2)).astype(np.int32)
#
#             if points.shape == (4, 2):  # Ensure correct shape
#                 cv2.polylines(frame, [points], isClosed=True, color=(0, 255, 0), thickness=2)
#             else:
#                 display("Warning: OBB points have incorrect shape:", points.shape)
#
#             # Draw label
#             conf = obb.conf[0].item()
#             cls = int(obb.cls[0].item())
#             label = f"{model.names[cls]} {conf:.2f}"
#             cv2.putText(frame, label, (points[0][0], points[0][1] - 10),
#                         cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
#
#     cv2.imshow("YOLO OBB Detection", frame)
#
#     # Press 'q' to exit
#     if cv2.waitKey(1) & 0xFF == ord("q"):
#         break
#
# cap.release()
# cv2.destroyAllWindows()

In [4]:
# cap.release()
# cv2.destroyAllWindows()

In [5]:
import cv2
def get_available_cameras():
    available_cameras = []
    # Check for 5 cameras
    for i in range(5):
        cap = cv2.VideoCapture(i)
        if cap.isOpened():
            available_cameras.append(i)
            cap.release()
    return available_cameras

cameras = get_available_cameras()
if cameras:
    print("Available Cameras:", cameras)
else:
    print("No cameras found.")

2025-04-22 22:09:11.687 Python[3073:26396568] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Available Cameras: [0, 1, 2]


OpenCV: out device of bound (0-2): 3
OpenCV: camera failed to properly initialize!
OpenCV: out device of bound (0-2): 4
OpenCV: camera failed to properly initialize!


In [6]:
import cv2
import numpy as np

def draw_perspective_grid(image, corners, grid_size=8, color=(0, 0, 255), thickness=1, dev_mode=True):
    """
    Draws an 8x8 grid inside a quadrilateral defined by `corners` using perspective transform.

    Params:
    - image: original image (BGR)
    - corners: list of 4 (x, y) tuples, approx corners of a square
    - grid_size: number of divisions along each side (default 8)
    - color: line color for drawing
    - thickness: thickness of the grid lines
    """
    img = image.copy()
    h, w = image.shape[:2]
    square_size = 800  # working warped image size

    # Sort corners: top-left, top-right, bottom-right, bottom-left
    def sort_corners(pts):
        pts = np.array(pts, dtype=np.float32)
        s = pts.sum(axis=1)
        diff = np.diff(pts, axis=1)
        return np.array([
            pts[np.argmin(s)],      # top-left
            pts[np.argmin(diff)],   # top-right
            pts[np.argmax(s)],      # bottom-right
            pts[np.argmax(diff)]    # bottom-left
        ], dtype=np.float32)

    src = sort_corners(corners)
    dst = np.array([
        [0, 0],
        [square_size, 0],
        [square_size, square_size],
        [0, square_size]
    ], dtype=np.float32)

    # Perspective transform matrices
    M = cv2.getPerspectiveTransform(src, dst)
    M_inv = cv2.getPerspectiveTransform(dst, src)

    step = square_size // grid_size

    intersection_points = []

    # Compute all intersections in warped space
    for i in range(grid_size + 1):
        intersection_points.append([])
        for j in range(grid_size + 1):
            pt = np.array([[[j * step, i * step]]], dtype=np.float32)
            pt_orig = cv2.perspectiveTransform(pt, M_inv)[0][0]
            pt_orig_int = tuple(np.int32(pt_orig))
            cv2.drawMarker(image, pt_orig_int, (0, 255, 255), thickness=2)
            intersection_points[-1].append(pt_orig_int)

    if dev_mode:
        # Draw vertical lines in warped space, then inverse transform
        for i in range(1, grid_size):
            x = i * step
            pt1 = np.array([[[x, 0]]], dtype=np.float32)
            pt2 = np.array([[[x, square_size]]], dtype=np.float32)
            pt1_orig = cv2.perspectiveTransform(pt1, M_inv)[0][0]
            pt2_orig = cv2.perspectiveTransform(pt2, M_inv)[0][0]
            cv2.line(img,
                     tuple(np.int32(pt1_orig)),
                     tuple(np.int32(pt2_orig)),
                     color, thickness)

        # Draw horizontal lines in warped space, then inverse transform
        for i in range(1, grid_size):
            y = i * step
            pt1 = np.array([[[0, y]]], dtype=np.float32)
            pt2 = np.array([[[square_size, y]]], dtype=np.float32)
            pt1_orig = cv2.perspectiveTransform(pt1, M_inv)[0][0]
            pt2_orig = cv2.perspectiveTransform(pt2, M_inv)[0][0]
            cv2.line(img,
                     tuple(np.int32(pt1_orig)),
                     tuple(np.int32(pt2_orig)),
                     color, thickness)

        # Optionally draw outer border
        cv2.polylines(img, [np.int32(src)], isClosed=True, color=(255, 0, 0), thickness=2)

    warped = cv2.warpPerspective(img, M, (square_size, square_size))
    warped = cv2.resize(warped, (300, 300))
    cv2.imshow('warped', warped)

    return img, intersection_points

In [7]:
def get_perspective_piece(coords, corners):
    def sort_corners(pts):
        pts = np.array(pts, dtype=np.float32)
        s = pts.sum(axis=1)
        diff = np.diff(pts, axis=1)
        return np.array([
            pts[np.argmin(s)],      # top-left
            pts[np.argmin(diff)],   # top-right
            pts[np.argmax(s)],      # bottom-right
            pts[np.argmax(diff)]    # bottom-left
        ], dtype=np.float32)

    square_size = 800  # working warped image size
    src = sort_corners(corners)
    dst = np.array([
        [0, 0],
        [square_size, 0],
        [square_size, square_size],
        [0, square_size]
    ], dtype=np.float32)

    # Perspective transform matrices
    M = cv2.getPerspectiveTransform(src, dst)
    M_inv = cv2.getPerspectiveTransform(dst, src)

    x1, y1, x2, y2 = coords
    piece_centre = np.array([[[(x1 + x2) / 2 ,y2]]],dtype=np.float32)
    #print(piece_centre)
    piece_corner_transformed = cv2.perspectiveTransform(piece_centre, M)[0][0]
    #print(piece_corner_transformed)

    piece_corner_transformed[1] -= 10
    return cv2.perspectiveTransform(np.array([[piece_corner_transformed]],dtype= np.float32), M_inv)[0][0]


In [8]:
def check_any_bbox_overlap(bboxes):
    """
    Checks if any of the 4 bounding boxes overlap.

    Args:
        bboxes (list of tuple): List of 4 bounding boxes in (x_min, y_min, x_max, y_max) format.

    Returns:
        bool: True if any two boxes overlap, else False.
    """
    def boxes_overlap(box1, box2):
        x1_min, y1_min, x1_max, y1_max = box1
        x2_min, y2_min, x2_max, y2_max = box2

        # Check if there is no overlap
        if x1_max <= x2_min or x2_max <= x1_min:
            return False
        if y1_max <= y2_min or y2_max <= y1_min:
            return False

        return True

    # Check all pairs
    for i in range(len(bboxes)):
        for j in range(i + 1, len(bboxes)):
            if boxes_overlap(bboxes[i], bboxes[j]):
                return True

    return False

In [9]:
def get_board(intersection_points, perspective_pieces):
    board = []
    FEN_label = {
    "Pawn_w": "P", "Knight_w": "N", "Bishop_w": "B", "Rook_w": 'R', "Queen_w": 'Q', "King_w": 'K',
    "Pawn_b": "p", "Knight_b": "n", "Bishop_b": 'b', "Rook_b": 'r', "Queen_b": 'q', "King_b": 'k',
    }
    try:
        #print(intersection_points)
        for i in range(8):
            row = ''
            #empty_count = 0
            for j in range(8):
                top_left = Point(intersection_points[i][j])
                top_right = Point(intersection_points[i][j+1])
                bottom_right = Point(intersection_points[i+1][j])
                bottom_left = Point(intersection_points[i+1][j+1])
                #print(i,j,top_left, top_right, bottom_right, bottom_left)
                polygon = Polygon([top_left, top_right, bottom_right, bottom_left])
                for corner,label in perspective_pieces:
                    if polygon.contains(Point(corner)):
                        #print(label)
                        # if empty_count:
                        #     row += str(empty_count)
                        #     empty_count = 0
                        row += FEN_label[label]
                        break
                else:
                    row += '.'
                    #empty_count += 1
                #print(j)
            #print(i,row)
            board.append(row)

    except Exception as e:
        print(e)
        board = None
    return board





In [10]:
def start_stockfish(path):
    # Example usage
    # Replace 'stockfish' with the path to your Stockfish executable if it's not in your system's PATH
    engine = subprocess.Popen(
        [path],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=True,
        bufsize=1
    )

    # Example: Send the 'uci' command to initialize UCI mode
    engine.stdin.write('uci\n')
    engine.stdin.flush()

    # Read and print the engine's response
    while True:
        output = engine.stdout.readline()
        if output.strip() == 'uciok':
            print('Stockfish Initialized')
            break

    return engine

def stockfish_nextmove(engine, fen):
    # Set the position using the provided FEN
    engine.stdin.write(f'position fen {fen}\n')
    engine.stdin.flush()

    # Instruct Stockfish to search for the best move within a time limit (e.g., 1 second)
    engine.stdin.write('go movetime 500\n')
    engine.stdin.flush()

    best_move = None
    # Read lines until the best move is found
    empty_line_count = 0
    while True:
        line = engine.stdout.readline()
        if line == '':
            empty_line_count += 1

        if line.startswith('bestmove'):
            best_move = line.split()[1]
            break

        if empty_line_count > 100:
            best_move = None
            break

    # Terminate the Stockfish process


    return best_move




In [11]:
engine = start_stockfish('./stockfish/stockfish-macos-m1-apple-silicon')

Stockfish Initialized


In [12]:
print(stockfish_nextmove(engine, '8/8/8/8/8/8/8/7P'))

None


In [13]:
def augument_nextMove(frame, next_move,inx_points):
    image = frame.copy()
    init_coords = 8 - (int(next_move[1])-1), (ord(next_move[0])-97)
    #init_coords= 1,1
    final_coords = 8 - (int(next_move[3])-1), (ord(next_move[2])-97)
    #print(init_coords, final_coords)
    init_box = np.array([[
        inx_points[init_coords[0]][init_coords[1]],
        inx_points[init_coords[0]-1][init_coords[1]],
        inx_points[init_coords[0]-1][init_coords[1]+1],
        inx_points[init_coords[0]][init_coords[1]+1]
        ]],dtype= np.int32)#.reshape(-1,1,2)
    final_box = np.array([[
        inx_points[final_coords[0]][final_coords[1]],
        inx_points[final_coords[0]-1][final_coords[1]],
        inx_points[final_coords[0]-1][final_coords[1]+1],
        inx_points[final_coords[0]][final_coords[1]+1]
    ]],dtype= np.int32)#.reshape(-1,1,2)
    frame = cv2.fillPoly(frame, init_box, (255,255,255))
    frame = cv2.fillPoly(frame, final_box, (0,0,255))
    alpha = 0.5
    frame = cv2.addWeighted(frame, alpha, image, 1 - alpha, 0, image)

    return frame

In [14]:
import cv2
import numpy as np
from PIL import ImageFont, ImageDraw, Image
import platform

def find_unicode_font():
    system = platform.system()
    if system == "Windows":
        return "C:/Windows/Fonts/seguisym.ttf"
    elif system == "Darwin":
        return "/System/Library/Fonts/Supplemental/Arial Unicode.ttf"
    elif system == "Linux":
        return "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf"
    else:
        raise RuntimeError("Unsupported OS for font detection.")

def display_chess_fen(fen: str):
    piece_unicode = {
        'P': '♙', 'N': '♘', 'B': '♗', 'R': '♖', 'Q': '♕', 'K': '♔',
        'p': '♟', 'n': '♞', 'b': '♝', 'r': '♜', 'q': '♛', 'k': '♚',
    }

    board_size = 480
    square_size = board_size // 8
    light_square = (239, 239, 209)  # From uploaded image
    dark_square = (120, 140, 82)    # From uploaded image

    img = np.ones((board_size, board_size, 3), dtype=np.uint8) * 255
    img_pil = Image.fromarray(img)
    draw = ImageDraw.Draw(img_pil)

    font_path = find_unicode_font()
    font = ImageFont.truetype(font_path, square_size - 10)

    position = fen.split()[0]
    rows = position.split('/')

    for row_idx in range(8):
        for col_idx in range(8):
            x = col_idx * square_size
            y = row_idx * square_size
            color = light_square if (row_idx + col_idx) % 2 == 0 else dark_square
            draw.rectangle([x, y, x + square_size, y + square_size], fill=color)

    for row_idx, row in enumerate(rows):
        col_idx = 0
        for char in row:
            if char.isdigit():
                col_idx += int(char)
            else:
                piece = piece_unicode.get(char)
                if piece:
                    x = col_idx * square_size
                    y = row_idx * square_size
                    bbox = font.getbbox(piece)
                    w = bbox[2] - bbox[0]
                    h = bbox[3] - bbox[1]
                    draw.text(
                        (x + (square_size - w) / 2, y + (square_size - h) / 2 - 5),
                        piece,
                        font=font,
                        fill=(0, 0, 0)
                    )
                col_idx += 1

    img = np.array(img_pil)
    cv2.imshow("FEN Chess Board", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
import cv2
import torch
import numpy as np
dev_mode = False
# Load YOLO model (ensure you have the correct model path)
# Open webcam (0 for default, 1 for external camera)
cap = cv2.VideoCapture(1)
engine = start_stockfish('./stockfish/stockfish-macos-m1-apple-silicon')
frame_count = 0
prev_fen = ''
next_move = None
while cap.isOpened():
    
    start_time = time.time()
    ret, frame = cap.read()  # Read frame from webcam
    if not ret:
        break
    frame_count += 1

    # Run YOLO inference
    results = model.predict(frame, conf=0.3, device="mps", verbose=False)  # Adjust confidence threshold as needed

    board_corners = []
    board_bbox = []

    pieces = []
    for result in results:
        if result.boxes is None or len(result.boxes) == 0:
            continue  # Skip frame if no detections

        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())  # Convert tensor to integer numpy array

            # Draw bounding box


            # Draw label
            conf = box.conf[0].item()
            cls = int(box.cls[0].item())
            label = f"{model.names[cls]} {conf:.2f}"
            if 'board_corner' == model.names[cls]:
                board_corners.append(((x1+x2)/2, (y1+ y2)/2))
                board_bbox.append((x1, y1, x2, y2))

                cv2.drawMarker(frame, ((x1 + x2)// 2, (y1 + y2) //2), (0, 0, 255), 1,18,6)
                #cv2.putText(frame, label, (x1, y1 - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            else:
                pieces.append(((x1,y1), (x2,y2), label))

    intersection_points = []
    if len(board_bbox) == 4 and not check_any_bbox_overlap(board_bbox):
        frame, intersection_points = draw_perspective_grid(frame, board_corners,grid_size=8,dev_mode=dev_mode)
        if dev_mode:
            for row in intersection_points:
                #print(intersection_points)
                for i,j in row:
                    cv2.drawMarker(frame, (i, j), (0, 255, 255), 2)

    perspective_pieces = []
    for (x1,y1),(x2,y2),label in pieces:
        trans_piece = list(map(int,get_perspective_piece((x1,y1,x2,y2),board_corners)))
        perspective_pieces.append([trans_piece,label.split()[0][:-1]])
        if dev_mode:
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            cv2.drawMarker(frame,list(trans_piece),(255, 255, 255), markerType=2, markerSize=15, thickness=1)

    if intersection_points:
        board = get_board(intersection_points, perspective_pieces)
        #print(board)
        if board:
            FEN = ''
            for row in board:
                empty = 0
                #print(row)
                for col in row+' ':
                    #print(col)
                    if col != '.':
                        if empty != 0:
                            FEN += str(empty)
                            empty = 0
                        if col != ' ':
                            FEN += col
                    else:
                        empty += 1

                FEN += '/'
            FEN = FEN[:-1]

            clear_output()
            board = chess.Board(FEN)
            custom_colors = {
                "square light": "#eeeed2",  # White squares
                "square dark": "#769656",   # Green squares
            }
            # Generate the SVG with custom colors
            svg_board = chess.svg.board(board=board, colors=custom_colors,size = 350)

            display(SVG(svg_board),frame_count)

            display_chess_fen(FEN)
            if prev_fen != FEN:
                prev_fen = FEN
                next_move = stockfish_nextmove(engine, FEN)

            if next_move is None:
                engine = start_stockfish('./stockfish/stockfish-macos-m1-apple-silicon')
            else:
                frame = augument_nextMove(frame, next_move,intersection_points)

            cv2.putText(frame, next_move,(100,100),
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)

    #print(perspective_pieces)

    #print(frame.shape)
    frame = cv2.resize(frame, (2560,1440))
    # Show frame
    cv2.imshow("YOLO Detection", frame)
    end_time = time.time()
    display(f'Inference Time: {(end_time-start_time)*100:.4f} ms')
    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
engine.stdin.write('quit\n')
engine.stdin.flush()
engine.terminate()

2025-04-22 22:09:16.415 Python[3073:26396568] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-22 22:09:16.415 Python[3073:26396568] +[IMKInputSession subclass]: chose IMKInputSession_Modern
